In [38]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

PROJECT_ROOT = Path("..").resolve()
CORE7_DIR = PROJECT_ROOT / "artifact" / "core7"
CORE8_DIR = PROJECT_ROOT / "artifact" / "core8"

CORE8_DIR.mkdir(parents=True, exist_ok=True)

CORE7_EVENT_LOG = CORE7_DIR / "core7_governance_event_log.csv"
CORE8_STATE_TRACE = CORE8_DIR / "core8_refusal_state_trace.csv"
CORE8_EVENT_LOG = CORE8_DIR / "core8_refusal_event_log.csv"

assert CORE7_EVENT_LOG.exists(), "core7_governance_event_log.csv not found"

In [39]:
event_df = pd.read_csv(CORE7_EVENT_LOG)

REQUIRED_COLS = [
    "run_id", "case_id", "antibody_id", "step",
    "attempted", "action", "reason_code", "SoMS"
]

missing = [c for c in REQUIRED_COLS if c not in event_df.columns]
assert not missing, f"Missing required columns: {missing}"

df = event_df.copy()

# 파생 컬럼
df["blocked"] = df["action"].isin(["VETO", "FREEZE"])
df["veto"] = df["action"] == "VETO"
df["freeze"] = df["action"] == "FREEZE"

df = df.sort_values(["antibody_id", "case_id", "step"]).reset_index(drop=True)

df.head()

,run_id,case_id,antibody_id,step,attempted,action,reason_code,SoD,SoMS,conflict_flag,mutation_desc,intended_axis,blocked,veto,freeze
0,core7_1767774738,A_ALWAYS_ALLOW,antibody_A,0,True,ALLOW,NaN,0.3,0.0,False,CDR_mut_0,IMMUNO,False,False,False
1,core7_1767774738,A_ALWAYS_ALLOW,antibody_A,1,True,ALLOW,NaN,0.8,0.0,False,CDR_mut_5,VISCOSITY,False,False,False
2,core7_1767774738,A_ALWAYS_ALLOW,antibody_A,2,True,ALLOW,NaN,1.5,0.0,False,CDR_mut_13,VISCOSITY,False,False,False
3,core7_1767774738,A_ALWAYS_ALLOW,antibody_A,3,True,ALLOW,NaN,2.4,0.0,False,CDR_mut_10,IMMUNO,False,False,False
4,core7_1767774738,A_ALWAYS_ALLOW,antibody_A,4,True,ALLOW,NaN,4.4,0.0,True,CDR_mut_8,PROCESS,False,False,False


In [40]:
with open(POLICY_PATH, "r") as f:
    POLICY = json.load(f)

MIN_STEPS = POLICY["min_guard_conditions"]["MIN_STEPS_BEFORE_FALLBACK"]
MIN_ATTEMPTS = POLICY["min_guard_conditions"]["MIN_ATTEMPTS_WINDOW"]

# 누적 기준
ACC = POLICY["accumulative_thresholds"]

WINDOW_SIZE = POLICY["accumulative_thresholds"]["WINDOW_SIZE"]
MIN_STEPS = POLICY["min_guard_conditions"]["MIN_STEPS_BEFORE_FALLBACK"]

T1 = POLICY["accumulative_thresholds"]["BLOCKED_RATE_THRESHOLD_STAGE1"]
T2 = POLICY["accumulative_thresholds"]["BLOCKED_RATE_THRESHOLD_STAGE2"]
C1 = POLICY["accumulative_thresholds"]["VETO_STREAK_THRESHOLD"]
O1 = POLICY["accumulative_thresholds"]["OSCILLATION_THRESHOLD"]
S2 = POLICY["accumulative_thresholds"]["SOMS_CUMSUM_THRESHOLD"]

In [41]:
def refusal_transition(
    prev_stage,
    step,
    blocked_rate,
    veto_streak,
    toggle_rate,
    soms_cumsum,
    freeze_flag,
):
    if step < MIN_STEPS:
        return prev_stage, False, "REASON_MIN_STEPS_NOT_REACHED"

    # Stage 0 → 1
    if prev_stage == 0:
        if blocked_rate >= T1 or veto_streak >= C1:
            return 1, True, "REASON_BLOCKED_RATE_ACCUMULATION"

    # Stage 1 → 2
    if prev_stage == 1:
        if blocked_rate >= T2 or soms_cumsum >= S2 or toggle_rate >= O1:
            return 2, True, "REASON_STAGE_ESCALATION"

    # Stage 2 → 3
    if prev_stage == 2:
        if freeze_flag:
            return 3, True, "REASON_REFUSAL_TERMINATION"

    return prev_stage, False, ""

In [42]:
def compute_window_metrics(sub_df: pd.DataFrame) -> pd.DataFrame:
    """
    Core 8 refusal 판단을 위한 window 기반 누적 지표 계산
    """
    rows = []

    for i in range(len(sub_df)):
        start = max(0, i - WINDOW_SIZE + 1)
        window = sub_df.iloc[start:i+1]

        # blocked rate
        blocked_rate = window["blocked"].mean()

        # veto streak
        streak = 0
        for a in reversed(window["action"].tolist()):
            if a == "VETO":
                streak += 1
            else:
                break

        # action toggle rate (oscillation)
        actions = window["action"].tolist()
        toggles = sum(actions[j] != actions[j-1] for j in range(1, len(actions)))
        toggle_rate = toggles / max(1, len(actions) - 1)

        # SoMS cumulative
        soms_cumsum = window["SoMS"].sum()

        rows.append({
            "blocked_rate_window": blocked_rate,
            "veto_streak": streak,
            "action_toggle_rate": toggle_rate,
            "SoMS_cumsum_window": soms_cumsum,
        })

    return pd.DataFrame(rows)

In [43]:
state_rows = []
event_rows = []

for (ab, case), sub in df.groupby(["antibody_id", "case_id"]):
    sub = sub.reset_index(drop=True)
    metrics = compute_window_metrics(sub)

    stage = 0

    for i in range(len(sub)):
        row = sub.iloc[i]
        m = metrics.iloc[i]

        new_stage, triggered, reason = refusal_transition(
            stage,
            row["step"],
            m["blocked_rate_window"],
            m["veto_streak"],
            m["action_toggle_rate"],
            m["SoMS_cumsum_window"],
            row["freeze"],
        )

        mode = ["NORMAL", "RATE_LIMIT", "PARTIAL_SEAL", "REFUSAL"][new_stage]

        state_rows.append({
            "run_id": row["run_id"],
            "case_id": case,
            "antibody_id": ab,
            "step": row["step"],
            "refusal_stage": new_stage,
            "refusal_mode": mode,
            "blocked_rate_window": m["blocked_rate_window"],
            "veto_streak": m["veto_streak"],
            "action_toggle_rate": m["action_toggle_rate"],
            "SoMS_cumsum_window": m["SoMS_cumsum_window"],
            "refusal_triggered": triggered,
            "refusal_reason_code": reason,
        })

        if triggered and new_stage != stage:
            event_rows.append({
                "run_id": row["run_id"],
                "case_id": case,
                "antibody_id": ab,
                "step": row["step"],
                "from_stage": stage,
                "to_stage": new_stage,
                "reason_code": reason,
            })

        stage = new_stage

In [44]:
state_df = pd.DataFrame(state_rows)
event_df = pd.DataFrame(event_rows)

state_df.to_csv(CORE8_STATE_TRACE, index=False)
event_df.to_csv(CORE8_EVENT_LOG, index=False)

CORE8_STATE_TRACE, CORE8_EVENT_LOG

(PosixPath('/Users/mac/Desktop/De/Developability_Data/core/artifact/core8/core8_refusal_state_trace.csv'),
 PosixPath('/Users/mac/Desktop/De/Developability_Data/core/artifact/core8/core8_refusal_event_log.csv'))